In [1]:
import os
from scipy.optimize import least_squares, curve_fit
from stl import mesh
import math
import numpy as np
from matplotlib import pyplot, pyplot as plt
from mpl_toolkits import mplot3d
from tqdm.auto import tqdm

In [55]:
def get3DDistance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)

def get2DDistance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def residuals(params, points):
    a, b, R = params
    return np.sqrt((points[:, 0] - a)**2 + (points[:, 1] - b)**2) - R

def line(x, A, B):
    return A * x + B

def sinusoid(x, A, B, C, D):
    return A * np.sin(B * (x - C)) + D

def rotated_sinusoid(xy, A, B, C, D, theta):
    x, y = xy
    # Rotate the coordinates
    x_prime = x * np.cos(theta) + y * np.sin(theta)
    y_prime = -x * np.sin(theta) + y * np.cos(theta)
    
    # Apply the sinusoidal function in the rotated frame
    return A * np.sin(B * x_prime + C) + D

def tangent_curve(x, A, B, C, D):
    return A * np.tan(B * (x - C)) + D

def getView(model, axis):
    view = []
    for triangle in tqdm(model.vectors, desc='Making view'):
        for point in triangle:
            if axis == 2:
                view.append([point[0], point[1]])
            elif axis == 1:
                view.append([point[0], point[2]])
            elif axis == 0:
                view.append([point[1], point[2]])
    return np.array(view)

def getCircleApprox(points):
    # Initial guess for the circle's center (a, b) and radius R
    x_m = np.mean(points[:, 0])
    y_m = np.mean(points[:, 1])
    R_m = np.mean(np.sqrt((points[:, 0] - x_m)**2 + (points[:, 1] - y_m)**2))
    initial_guess = [x_m, y_m, R_m]
    
    # Perform the least squares optimization
    result = least_squares(residuals, initial_guess, args=(points,))
    
    a_opt, b_opt, R_opt = result.x
    
    return a_opt, b_opt, R_opt

def getSinusoidApprox(points, initial_guess):
    # Separate the points into x and y coordinates
    x_data = points[:, 0]
    y_data = points[:, 1]
    
    # initial_guess = [0.7, 1/20, 0, -11]
    
    # Perform the curve fitting
    params, params_covariance = curve_fit(sinusoid, x_data, y_data, p0=initial_guess)
    
    # Extract the optimal parameters
    A_opt, B_opt, C_opt, D_opt = params
    
    return A_opt, B_opt, C_opt, D_opt

def getTangentApprox(points, initial_guess):
    # Separate the points into x and y coordinates
    x_data = points[:, 1]
    y_data = points[:, 0]
    
    # initial_guess = [0.7, 1/20, 0, -11]
    
    # Perform the curve fitting
    params, params_covariance = curve_fit(tangent_curve, x_data, y_data, p0=initial_guess)
    
    # Extract the optimal parameters
    A_opt, B_opt, C_opt, D_opt = params
    
    return A_opt, B_opt, C_opt, D_opt

def getRotatedSinusoidApprox(points, initial_guess):
    # Separate the points into x and y coordinates
    x_data = points[:, 0]
    y_data = points[:, 1]
    
    # initial_guess = [0.7, 1/20, 0, -11]
    
    # Perform the curve fitting
    params, params_covariance = curve_fit(rotated_sinusoid, (x_data, y_data), y_data, p0=initial_guess, maxfev=50000)
    
    # Extract the optimal parameters
    A_opt, B_opt, C_opt, D_opt, theta_opt = params
    
    return A_opt, B_opt, C_opt, D_opt, theta_opt

def getLineApprox(points, initial_guess):
    x_data = points[:, 0]
    y_data = points[:, 1]
    params, params_covariance = curve_fit(line, x_data, y_data, p0=initial_guess)
    A, B = params
    return A, B

def getAngle(center, beginning, coord):
    a = get2DDistance(center, beginning)
    b = get2DDistance(center, coord)
    c = get2DDistance(beginning, coord)
    val = (a*a + b*b - c*c) / (2*a*b)
    angle = 0
    if val < -1:
        print(val)
        val = val+2
        angle = math.pi
    if val > 1:
        print(val)
        val = val-2
        angle = -math.pi
    angle += math.acos(val)
    if coord[0] > beginning[0]:
        angle = 2*math.pi - angle
    return angle

def transformCoordinate(coord, center, beginning, radius):
    # print(coord, center, beginning, radius)
    angle = getAngle(center, beginning, coord)
    if angle is not None:
        return [angle * radius, radius - get2DDistance(center, coord), coord[2]]
    else:
        return None
    
def getVector(A, B):
    return [B[0] - A[0], B[1] - A[1], B[2] - A[2]]
    
def getCenterFromLine(A, B, z):
    v = getVector(A, B)
    t = (z - A[2]) / v[2]
    return A[0] + t * v[0], A[1] + t * v[1], A[2] + t * v[2]

def transform3DModel(model, layer_from, layer_mid, layer_to, sin_a, sin_b, sin_c, sin_d):
    model_layer = getModelLayer(model, 2, layer_from, layer_mid)
    # print(model_layer)
    view = getView(model_layer, 2)
    # print(view.shape)
    x, y, radius = getCircleApprox(view)
    center_a = [x, y, (5+2)/2]
    print(f"Lower part center: {center_a} and radius: {radius}")
    
    model_layer = getModelLayer(model, 2, layer_mid, layer_to)
    # print(model_layer)
    view = getView(model_layer, 2)
    # print(view.shape)
    x, y, _ = getCircleApprox(view)
    
    center_b = [x, y, (63+5)/2]
    print(f"Thread part center: {center_b}")
    # center = getCenter(model)
    # centers = getCenters(model)
    # center = centers[np.where(centers[:, 2] == point[2])][0]
    # radius = getRadius(model)
    data_transformed = np.zeros(model.data.shape[0], dtype=mesh.Mesh.dtype)
    for i, triangle in tqdm(enumerate(model.data), total=model.data.shape[0], desc=f"Transforming 3D model"):
        vectors = []
        for point in triangle[1]:
            center = getCenterFromLine(center_a, center_b, point[2])
            # print(center)
            vectors.append(transformCoordinate(point, center, getBeginning(radius, center), radius))
        if None not in vectors and np.std(vectors, axis=0)[0] < 10:
            data_transformed['vectors'][i] = np.array(vectors)
            
    return mesh.Mesh(data_transformed)
    
def removeSinusoidErrorY(model, layer_from, layer_mid, layer_to, sin_a, sin_b, sin_c, sin_d):
    print("Starting to remove sinusoid error on Y axis")
    model_layer = getModelLayer(model, 2, layer_from, layer_mid)
    view = getView(model_layer, 2)
    A, B = getLineApprox(view, [-0.001, 0])
    print(f"Line approx params: {A, B}")
    for i, coord in enumerate(view):
        view[i, 1] -= line(view[i, 0], A, B)
    
    A, B, C, D = getSinusoidApprox(view, [sin_a, sin_b, sin_c, sin_d])
    print(f"Sinusoid approx params: {A, B, C, D}")
    
    data_transformed = np.zeros(model.data.shape[0], dtype=mesh.Mesh.dtype)
    for i, triangle in tqdm(enumerate(model.data), total=model.data.shape[0], desc=f"Removing Sinusoid Error on Y") :
        vectors = []
        for point in triangle[1]:
            vectors.append([point[0], point[1]-sinusoid(point[0], A, B, C, D), point[2]])
        data_transformed['vectors'][i] = np.array(vectors)
    return mesh.Mesh(data_transformed)

def removeSinusoidErrorZ(model, layer_from, layer_to, sin_a, sin_b, sin_c, sin_d, left_up, right_down):
    print("Starting to remove sinusoid error on Z axis")
    model_layer = getModelLayer(unrolled_scan, 2, layer_from, layer_to)
    model_layer = getModelLayer(model_layer, 1, 0, 2)
    view = getView(model_layer, 1)
    new_view = []
    for coord in view:
        if not((coord[0] < left_up[0] and coord[1] > left_up[1]) or (coord[0] > right_down[0] and coord[1] < right_down[1])):
            new_view.append(coord)
    view = np.array(new_view)
    A, B = getLineApprox(view, [0.001, 14])
    print(f"Line approx params: {A, B}")
    for i, coord in enumerate(view):
        view[i, 1] -= line(view[i, 0], A, B)
    view = np.round(view, 1)
    unique_x = np.unique(view[:, 0])
    avg_y_for_x = np.array([
        [x, np.mean(view[view[:, 0] == x, 1])]
        for x in unique_x
    ])
    A, B, C, D = getSinusoidApprox(avg_y_for_x, [sin_a, sin_b, sin_c, sin_d])
    print(f"Sinusoid approx params: {A, B, C, D}")
    
    data_transformed = np.zeros(model.data.shape[0], dtype=mesh.Mesh.dtype)
    for i, triangle in tqdm(enumerate(model.data), total=model.data.shape[0], desc=f"Removing Sinusoid Error on Z"):
        vectors = []
        for point in triangle[1]:
            vectors.append([point[0], point[1], point[2]-sinusoid(point[0], A, B, C, D)])
        data_transformed['vectors'][i] = np.array(vectors)
    return mesh.Mesh(data_transformed)

def visualizeModel(model):
    # Create a new plot
    figure = pyplot.figure(figsize=(15, 15))
    axes = figure.add_subplot(projection='3d')
    axes.set_xlabel('X')
    axes.set_ylabel('Y')
    axes.set_zlabel('Z')
    
    # Render the cube
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(model.vectors))
    
    # Auto scale to the mesh size
    scale = model.points.flatten()
    axes.auto_scale_xyz(scale, scale, scale)
    
    # Show the plot to the screen
    pyplot.show()
    
def getModelLayer(model, axis, from_, to):
    vertices = model.vectors  # shape (n, 3, 3) where n is the number of triangles
    centroids = np.mean(vertices, axis=1)  # shape (n, 3)
    mask = (centroids[:, axis] >= from_) & (centroids[:, axis] <= to)
    data = np.zeros(vertices[mask].shape[0], dtype=mesh.Mesh.dtype)
    data['vectors'] = vertices[mask]
    return mesh.Mesh(data)
    
    
def visualizeModelLayer(model, axis, from_, to):
    
    vertices = model.vectors  # shape (n, 3, 3) where n is the number of triangles
    centroids = np.mean(vertices, axis=1)  # shape (n, 3)
    mask = (centroids[:, axis] >= from_) & (centroids[:, axis] <= to)
    filtered_vertices = vertices[mask]
    
    
    # Create a new plot
    figure = pyplot.figure(figsize=(15, 15))
    axes = figure.add_subplot(projection='3d')
    axes.set_xlabel('X')
    axes.set_ylabel('Y')
    axes.set_zlabel('Z')
    
    # Render the cube
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(filtered_vertices))
    
    # Auto scale to the mesh size
    scale = model.points.flatten()
    axes.auto_scale_xyz(scale, scale, scale)
    
    # Show the plot to the screen
    pyplot.show()

def getCenter(model):
    vertices = model.vectors  
    
    centroids = np.mean(vertices, axis=1)  
    
    cross_products = np.cross(vertices[:, 1] - vertices[:, 0], vertices[:, 2] - vertices[:, 0])
    areas = np.linalg.norm(cross_products, axis=1) / 2  
    
    weighted_sum = np.sum(centroids * areas[:, np.newaxis], axis=0)  
    
    total_area = np.sum(areas)
    
    center_of_mass = weighted_sum / total_area
    
    return center_of_mass

def getCenterAlt(model, axis, from_, to):
    vertices = model.vectors  # shape (n, 3, 3) where n is the number of triangles
    centroids = np.mean(vertices, axis=1)  # shape (n, 3)
    mask = (centroids[:, axis] >= from_) & (centroids[:, axis] <= to)
    filtered_centroids = centroids[mask]
    filtered_vertices = vertices[mask]
    
    cross_products = np.cross(filtered_vertices[:, 1] - filtered_vertices[:, 0], filtered_vertices[:, 2] - filtered_vertices[:, 0])
    areas = np.linalg.norm(cross_products, axis=1) / 2  # shape (n,)
    
    # Calculate the weighted sum of centroids
    weighted_sum = np.sum(filtered_centroids * areas[:, np.newaxis], axis=0)  # shape (3,)
    
    # Calculate the total area
    total_area = np.sum(areas)
    
    # Compute the center of mass
    center_of_mass = weighted_sum / total_area
    
    return center_of_mass

def getCenters(model):
    # Extract the vertices and their z-coordinates
    vertices = model.vectors.reshape(-1, 3)
    z_coords = vertices[:, 2]
    
    # Identify the unique z-coordinates and their indices
    unique_z_coords, inverse_indices = np.unique(z_coords, return_inverse=True)
    
    # Initialize arrays to store sums and counts for each layer
    sums_x = np.zeros(len(unique_z_coords))
    sums_y = np.zeros(len(unique_z_coords))
    counts = np.zeros(len(unique_z_coords))
    
    # Sum the x and y coordinates and count the number of points in each layer
    np.add.at(sums_x, inverse_indices, vertices[:, 0])
    np.add.at(sums_y, inverse_indices, vertices[:, 1])
    np.add.at(counts, inverse_indices, 1)
    
    # Calculate the center of mass for each layer
    com_x = sums_x / counts
    com_y = sums_y / counts
    com_z = unique_z_coords
    
    # Combine the results into a single array
    centers_of_mass = np.vstack((com_x, com_y, com_z)).T
    
    return centers_of_mass

def getBeginning(radius, center):
    return [center[0], center[1] + radius, center[2]]
    
def getYMax(model):
    loc = np.argmax(model.vectors[:, :, 1])
    return model.vectors[loc//3][loc%3]

def getYMin(model):
    loc = np.argmin(model.vectors[:, :, 1])
    return model.vectors[loc//3][loc%3]

def getXMax(model):
    loc = np.argmax(model.vectors[:, :, 0])
    return model.vectors[loc//3][loc%3]

def getXMin(model):
    loc = np.argmin(model.vectors[:, :, 0])
    return model.vectors[loc//3][loc%3]

def getZMax(model):
    loc = np.argmax(model.vectors[:, :, 2])
    return model.vectors[loc//3][loc%3]

def getZMin(model):
    loc = np.argmin(model.vectors[:, :, 2])
    return model.vectors[loc//3][loc%3]

def getRadius(model):
    vertices = model.vectors  # shape (n, 3, 3) where n is the number of triangles
    # centroids = np.mean(vertices, axis=1)  # shape (n, 3)
    # mask = (centroids[:, 2] >= 15) & (centroids[:, 2] <= 70)
    # vertices = vertices[mask]
    loc_y_max = np.argmax(vertices[:, :, 1])
    loc_x_max = np.argmax(vertices[:, :, 0])
    loc_x_min = np.argmin(vertices[:, :, 0])
    loc_y_min = np.argmin(vertices[:, :, 1])
    # print(loc_x_max, loc_x_min, loc_y_max, loc_y_min)
    
    return (get2DDistance(model.vectors[loc_y_min//3][loc_y_min%3], model.vectors[loc_y_max//3][loc_y_max%3]) + get2DDistance(model.vectors[loc_x_min//3][loc_x_min%3], model.vectors[loc_x_max//3][loc_x_max%3]))/4

In [3]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
folders = [file for file in os.listdir(SRC) if file.startswith("H_")]
scans = []
for folder in folders:
    scans.append(
        os.path.join(folder, [file for file in os.listdir(os.path.join(SRC, folder)) if file.endswith(".stl")][0]))

In [4]:
scan_models = []
for scan in scans:
    scan_models.append(mesh.Mesh.from_file(os.path.join(SRC, scan)))

In [67]:
pipes = [[True, True, True], 
        [True, True, True],
         [True, False, True], 
         [False, True, True],
         [True, True, True],
         [True, True, True]]

In [68]:
params = [[dict(layer_from=2,
                layer_mid=5,
                layer_to=63,
                sin_a=1/10,
                sin_b=1/20,
                sin_c=0,
                sin_d=0), 
           dict(layer_from=14,
                layer_to=22,
                sin_a=1/5,
                sin_b=1/40,
                sin_c=0,
                sin_d=0,
                left_up=(100, 19), 
                right_down=(125, 16))],
          [dict(layer_from=3,
               layer_mid=5,
               layer_to=63,
               sin_a=1/10,
               sin_b=1/16,
               sin_c=0,
               sin_d=0), 
           dict(layer_from=14,
                layer_to=22,
                sin_a=1/5,
                sin_b=1/40,
                sin_c=0,
                sin_d=0,
                left_up=(100, 19), 
                right_down=(125, 16))],
          [dict(layer_from=3,
               layer_mid=5,
               layer_to=63,
               sin_a=1/10,
               sin_b=1/16,
               sin_c=75,
               sin_d=0), 
           dict(layer_from=14,
                layer_to=22,
                sin_a=1/5,
                sin_b=1/40,
                sin_c=0,
                sin_d=0,
                left_up=(100, 19), 
                right_down=(125, 16))],
          [dict(layer_from=5,
               layer_mid=7,
               layer_to=60,
               sin_a=1/10,
               sin_b=1/16,
               sin_c=0,
               sin_d=0), 
           dict(layer_from=14,
                layer_to=22,
                sin_a=1/5,
                sin_b=1/40,
                sin_c=0,
                sin_d=0,
                left_up=(100, 19), 
                right_down=(125, 16))],
          [dict(layer_from=5,
               layer_mid=6,
               layer_to=55,
               sin_a=1/10,
               sin_b=1/16,
               sin_c=0,
               sin_d=0), 
           dict(layer_from=12,
                layer_to=20,
                sin_a=1/5,
                sin_b=1/40,
                sin_c=0,
                sin_d=0,
                left_up=(50, 18), 
                right_down=(125, 14))],
          [dict(layer_from=6,
               layer_mid=7,
               layer_to=60,
               sin_a=1/10,
               sin_b=1/16,
               sin_c=0,
               sin_d=0), 
           dict(layer_from=14,
                layer_to=22,
                sin_a=1/5,
                sin_b=1/40,
                sin_c=0,
                sin_d=0,
                left_up=(75, 20), 
                right_down=(100, 17))]]

In [69]:
for scan_model, scan, param, pipe in zip(scan_models, scans, params, pipes):
    if not os.path.isfile(os.path.join(SRC, scan).replace('.stl', '_unrolled_final.stl')):
        print(f"Transforming scan: {scan}")
        if os.path.isfile(os.path.join(SRC, scan).replace('.stl', '_unrolled.stl')):
            unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scan).replace('.stl', '_unrolled.stl'))
        else:
            unrolled_scan = transform3DModel(scan_model, **param[0])
            unrolled_scan.save(os.path.join(SRC, scan).replace('.stl', '_unrolled.stl'))
            
        if pipe[0]:
            if os.path.isfile(os.path.join(SRC, scan).replace('.stl', '_unrolled_sin_y.stl')):
                unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scan).replace('.stl', '_unrolled_sin_y.stl'))
            else:
                unrolled_scan = removeSinusoidErrorY(unrolled_scan, **param[0])
                unrolled_scan.save(os.path.join(SRC, scan).replace('.stl', '_unrolled_sin_y.stl'))
                
        if pipe[1]:    
            if os.path.isfile(os.path.join(SRC, scan).replace('.stl', '_unrolled_sin_y_z.stl')):
                unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scan).replace('.stl', '_unrolled_sin_y_z.stl'))
            else:
                unrolled_scan = removeSinusoidErrorZ(unrolled_scan, **param[1])
                unrolled_scan.save(os.path.join(SRC, scan).replace('.stl', '_unrolled_sin_y_z.stl'))
            
        if pipe[2]:
            if os.path.isfile(os.path.join(SRC, scan).replace('.stl', '_unrolled_final.stl')):
                unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scan).replace('.stl', '_unrolled_final.stl'))
            else:
                unrolled_scan = getModelLayer(unrolled_scan, 1, -1, 6.2)
                unrolled_scan.save(os.path.join(SRC, scan).replace('.stl', '_unrolled_final.stl'))

In [ ]:
visualizeModel(scan_model)

In [36]:
getCenter(scan_model)

In [62]:
getCenterAlt(scan_model, 2, 2, 7)

In [66]:
getRadius(scan_model)

In [14]:
getXMax(scan_model),getXMin(scan_model)

In [15]:
getYMax(scan_model),getYMin(scan_model)

In [18]:
getZMax(scan_model),getZMin(scan_model)

In [20]:
loc = np.argmin(scan_model.vectors[:, :, 2])

In [21]:
loc//3

In [31]:
scan_model.vectors[scan_model.vectors[:, :, 2] > 80]

In [24]:
# getXMax(scan_model)

In [46]:
np.std(scan_model.vectors[0], axis=0)[0]

In [11]:
coms = getCenters(scan_model)

In [20]:
coms[0][2]

In [30]:
coms[np.where(coms[:, 2] == -0.6549979448318481)][0]

In [26]:
np.mean(coms, axis=0), np.std(coms, axis=0)

In [39]:
scan_model = mesh.Mesh.from_file(os.path.join(SRC, scans[0]))
unrolled_scan = transform3DModel(scan_model)
unrolled_scan.save(os.path.join(SRC, scans[0]).replace('.stl', '_unrolledv4.stl'))

In [169]:
scans[3]

In [40]:
unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scans[5]).replace('.stl', '_unrolled.stl'))

In [42]:
model_layer = getModelLayer(unrolled_scan, 2, 6, 7)
model_layer = getModelLayer(model_layer, 1, -1, 1)

In [43]:
view = getView(model_layer, 2)

In [44]:
plt.scatter(view[:, 0], view[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [45]:
A, B = getLineApprox(view, [-0.001, 0])

In [46]:
A, B

In [47]:
for i, coord in enumerate(view):
    view[i, 1] -= line(view[i, 0], A, B)

In [48]:
plt.scatter(view[:, 0], view[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [51]:
getSinusoidApprox(view, [1/10, 1/16, 0, 0])

In [232]:
removeSinusoidErrorY(unrolled_scan, [1/10, 1/20, 25, 0]).save(os.path.join(SRC, scans[0]).replace('.stl', '_unrolled_siny.stl'))

In [31]:
scans[4]

In [59]:
unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scans[5]).replace('.stl', '_unrolled_sin_y.stl'))

In [63]:
model_layer = getModelLayer(unrolled_scan, 2, 14, 22)
model_layer = getModelLayer(model_layer, 1, 0, 2)

In [64]:
view = getView(model_layer, 1)

In [65]:
plt.scatter(view[:, 0], view[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [17]:
new_view = []
for coord in view:
    if not((coord[0] <= 50 and coord[1] > 18) or (coord[0] > 125 and coord[1] < 14)):
        new_view.append(coord)
view = np.array(new_view)

In [18]:
plt.scatter(view[:, 0], view[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [19]:
A, B = getLineApprox(view, [0.1, 18])

In [20]:
A,B

In [21]:
for i, coord in enumerate(view):
    view[i, 1] -= line(view[i, 0], A, B)

In [22]:
getLineApprox(view, [0, 14])

In [23]:
plt.scatter(view[:, 0], view[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [24]:
view = np.round(view, 1)

In [25]:
view.shape

In [26]:
unique_x = np.unique(view[:, 0])

In [27]:
unique_x.shape

In [28]:
avg_y_for_x = np.array([
    [x, np.mean(view[view[:, 0] == x, 1])]
    for x in unique_x
])

In [29]:
plt.scatter(avg_y_for_x[:, 0], avg_y_for_x[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [30]:
getSinusoidApprox(avg_y_for_x, [0.2, 1/40, 0, 0])

In [344]:
removeSinusoidErrorZ(unrolled_scan, [0.5, 1/40, 110, 0]).save(os.path.join(SRC, scans[0]).replace('.stl', '_unrolled_siny_sinz.stl'))

In [6]:
unrolled_scan = mesh.Mesh.from_file(os.path.join(SRC, scans[0]).replace('.stl', '_unrolled_siny_sinz.stl'))

In [8]:
visualizeModelLayer(unrolled_scan, 1, 6, 22)

In [15]:
getModelLayer(unrolled_scan, 1, 0, 6.2).save(os.path.join(SRC, scans[0]).replace('.stl', '_unrolled_final.stl'))

In [9]:
scan = mesh.Mesh.from_file(os.path.join(SRC, scans[5]))

In [34]:
scan_layer = getModelLayer(scan, 2, 7, 12)

In [35]:
view = getView(scan_layer, 2)

In [36]:
plt.scatter(view[:, 0], view[:, 1], s=1)
# plt.axis('equal')
plt.show()

In [207]:
getCircleApprox(view)